In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from scipy.stats import zscore


import pickle
import json

In [2]:
with open('x_y.model.pkl', 'rb') as f:
    x_loaded, y_loaded= pickle.load(f)



print('x', x_loaded)
print('*'*20)
print('y', y_loaded)

x       gender  SeniorCitizen  Partner  Dependents    tenure  PhoneService  \
0        0.0            0.0      1.0         0.0  0.013889           0.0   
1        1.0            0.0      0.0         0.0  0.472222           1.0   
2        1.0            0.0      0.0         0.0  0.027778           1.0   
3        1.0            0.0      0.0         0.0  0.625000           0.0   
4        0.0            0.0      0.0         0.0  0.027778           1.0   
...      ...            ...      ...         ...       ...           ...   
7038     1.0            0.0      1.0         1.0  0.333333           1.0   
7039     0.0            0.0      1.0         1.0  1.000000           1.0   
7040     0.0            0.0      1.0         1.0  0.152778           0.0   
7041     1.0            1.0      1.0         0.0  0.055556           1.0   
7042     1.0            0.0      0.0         0.0  0.916667           1.0   

      MultipleLines  OnlineSecurity  OnlineBackup  DeviceProtection  ...  \
0        

### Model Development 

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

x_train, x_test, y_train, y_test= train_test_split(x_loaded, y_loaded, test_size= 0.2, random_state=22, stratify= y_loaded)
y_train.value_counts()

Churn
0    4139
1    1495
Name: count, dtype: int64

In [4]:
y_loaded.value_counts()

Churn
0    5174
1    1869
Name: count, dtype: int64

In [5]:
print(4139/ (4139 + 1495))  #y_train_values

print(5174 / (5174+1869)) 
# here perfectly fitted data not change during curtain values chanes count of y on if we random_state changed

0.7346467873624423
0.7346301292063041


In [6]:
print(y_test.value_counts())
(1035/ (1035+ 374))

Churn
0    1035
1     374
Name: count, dtype: int64


0.7345635202271115

In [7]:
rf_clf= RandomForestClassifier(n_estimator= 100, criterion= 'entropy', max_depth= None, min_samples_split = 4, 
                              min_samples_leaf= 2, max_features= 'log2', bootstrap = True, oob_score= True, 
                              n_jobs= -1, random_state=22, verbose= 0)

rf_clf.fit(x_train, y_train)

DecisionTreeClassifier(random_state=22)

In [8]:
# This is imbalnced data training evaluations

y_pred_train=rf_clf.predict(x_train)
cnf_matrix=confusion_matrix(y_train, y_pred_train)
print("confusion Matrix of Training Evaluations:\n", cnf_matrix)
print("*"* 40)
accuracy=accuracy_score(y_train, y_pred_train)
print("Accuracy:", accuracy )
print("*"*20)
clf_report=classification_report(y_train, y_pred_train)
print("Classificaiton report:\n", clf_report)

confusion Matrix of Training Evaluations:
 [[4139    0]
 [  13 1482]]
****************************************
Accuracy: 0.9976925807596734
********************
Classificaiton report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4139
           1       1.00      0.99      1.00      1495

    accuracy                           1.00      5634
   macro avg       1.00      1.00      1.00      5634
weighted avg       1.00      1.00      1.00      5634



In [9]:
# This is imbalnced data testing evaluations

y_pred_test=rf_clf.predict(x_test)
cnf_matrix=confusion_matrix(y_test, y_pred_test)
print('Confusion Matrix of Testing Evaltuations: \n', cnf_matrix)

print("*"*40)
accuracy=accuracy_score(y_test, y_pred_test)
print("Accuracy:", accuracy)
print("*"*40)

clf_report=classification_report(y_test, y_pred_test)
print("classification report:\n", clf_report)

Confusion Matrix of Testing Evaltuations: 
 [[855 180]
 [184 190]]
****************************************
Accuracy: 0.7416607523066004
****************************************
classification report:
               precision    recall  f1-score   support

           0       0.82      0.83      0.82      1035
           1       0.51      0.51      0.51       374

    accuracy                           0.74      1409
   macro avg       0.67      0.67      0.67      1409
weighted avg       0.74      0.74      0.74      1409



In [10]:
y_loaded.value_counts()    # Thats are >> df['Churn']

Churn
0    5174
1    1869
Name: count, dtype: int64

#### Sampling Techniques

In [11]:
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.preprocessing import StandardScaler, MinMaxScaler

normal_scaler= MinMaxScaler()
array= normal_scaler.fit_transform(x_loaded)   #Scalling
x=pd.DataFrame(array, columns=x_loaded.columns)


random_oversampler= RandomOverSampler(sampling_strategy = 0.8, random_state= 22)
x_sampled, y_sampled = random_oversampler.fit_resample(x_loaded, y_loaded)
print(y_sampled.value_counts())
x_train_, x_test_, y_train_, y_test_ = train_test_split(x_sampled, y_sampled, test_size= 0.2, random_state=22)
print(x_loaded.shape)

print(y_loaded.shape)
print('*'*20)
rf_clf_= RandomForestClassifier(n_estimator= 100, criterion= 'entropy', max_depth= None, min_samples_split= 2, 
                               min_samples_leaf= 1, max_features= 'log2', bootstrap= True, oob_score= True,
                                n_jobs= -1, random_state= 22, verbose= 0)
rf_clf_.fit(x_train_, y_train_)

Churn
0    5174
1    4139
Name: count, dtype: int64
(7043, 26)
(7043,)
********************


DecisionTreeClassifier(random_state=22)

In [12]:
# This is balnced data training evaluations

y_pred_train=rf_clf_.predict(x_train_)
cnf_matrix=confusion_matrix(y_train_, y_pred_train)
print("confusion Matrix of Training Evaluations:\n", cnf_matrix)
print("*"* 40)
accuracy=accuracy_score(y_train_, y_pred_train)
print("Accuracy:", accuracy )
print("*"*20)
clf_report=classification_report(y_train_, y_pred_train)
print("Classificaiton report:\n", clf_report)

confusion Matrix of Training Evaluations:
 [[4147    8]
 [   9 3286]]
****************************************
Accuracy: 0.9977181208053691
********************
Classificaiton report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4155
           1       1.00      1.00      1.00      3295

    accuracy                           1.00      7450
   macro avg       1.00      1.00      1.00      7450
weighted avg       1.00      1.00      1.00      7450



In [13]:
# This is balnced data testing evaluations

y_pred_test=rf_clf_.predict(x_test_)
cnf_matrix=confusion_matrix(y_test_, y_pred_test)
print('Confusion Matrix of Testing Evaltuations: \n', cnf_matrix)

print("*"*40)
accuracy=accuracy_score(y_test_, y_pred_test)
print("Accuracy:", accuracy)
print("*"*40)

clf_report=classification_report(y_test_, y_pred_test)
print("classification report:\n", clf_report)

Confusion Matrix of Testing Evaltuations: 
 [[810 209]
 [ 91 753]]
****************************************
Accuracy: 0.8389694041867954
****************************************
classification report:
               precision    recall  f1-score   support

           0       0.90      0.79      0.84      1019
           1       0.78      0.89      0.83       844

    accuracy                           0.84      1863
   macro avg       0.84      0.84      0.84      1863
weighted avg       0.85      0.84      0.84      1863



In [15]:
# Feature Scalling Added
from sklearn.preprocessing import StandardScaler, MinMaxScaler


normal_scaler= MinMaxScaler()
array2= normal_scaler.fit_transform(x_loaded)
x=pd.DataFrame(array2, columns= x_loaded.columns )

smote= SMOTE(sampling_strategy = 0.8, k_neighbors= 35) #Backed Run KNeighbor Algorithms
x_sampled_, y_sampled_ =smote.fit_resample(x_loaded, y_loaded)
print(y_sampled.value_counts())
print('*'*20)
x_train__, x_test__, y_train__, y_test__= train_test_split(x_sampled_, y_sampled_, test_size=0.2, random_state= 22)
print(x_loaded.shape)
print(y_loaded.shape)
print('*'*20)

rf_clf__= RandomForestClassifier(n_estimator= 100, criterion= 'entropy', max_depth= None, min_samples_split= 2, 
                               min_samples_leaf= 1, max_features= 'log2', bootstrap= True, oob_score= True,
                                n_jobs= -1, random_state= 22, verbose= 0)
rf_clf__.fit(x_train_, y_train_)


print('*'* 20)


# This is balnced data training evaluations

y_pred_train=rf_clf__.predict(x_train__)
cnf_matrix=confusion_matrix(y_train__, y_pred_train)
print("confusion Matrix of Training Evaluations:\n", cnf_matrix)
print("*"* 40)
accuracy=accuracy_scaore(y_train__, y_pred_train)
print("Accuracy:", accuracy )
print("*"*20)
clf_report=classification_report(y_train__, y_pred_train)
print("Classificaiton report:\n", clf_report)

# This is balnced data testing evaluations

y_pred_test=rf_clf__.predict(x_test__)
cnf_matrix=confusion_matrix(y_test__, y_pred_test)
print('Confusion Matrix of Testing Evaltuations: \n', cnf_matrix)

print("*"*40)
accuracy=accuracy_score(y_test__, y_pred_test)
print("Accuracy:", accuracy)
print("*"*40)

clf_report=classification_report(y_test__, y_pred_test)
print("classification report:\n", clf_report)

Churn
0    5174
1    4139
Name: count, dtype: int64
********************
(7043, 26)
(7043,)
********************
********************
confusion Matrix of Training Evaluations:
 [[4154    1]
 [  11 3284]]
****************************************
Accuracy: 0.9983892617449664
********************
Classificaiton report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4155
           1       1.00      1.00      1.00      3295

    accuracy                           1.00      7450
   macro avg       1.00      1.00      1.00      7450
weighted avg       1.00      1.00      1.00      7450

Confusion Matrix of Testing Evaltuations: 
 [[798 221]
 [232 612]]
****************************************
Accuracy: 0.7568438003220612
****************************************
classification report:
               precision    recall  f1-score   support

           0       0.77      0.78      0.78      1019
           1       0.73      0.73      0.73

#### Feature Importance

In [ ]:
s1=pd.Series(rf_clf.feature_importances_, index= x_loaded.columns)
s1.sort_values().plot(kind= 'barh')

#### Out Of Bag Samples Score

In [ ]:
rf_clf.oob_score

### Hyper-Tunning 

In [25]:
rf__clf= RandomForestClassifier(n_estimator= 100, criterion= 'entropy', max_depth= None, min_samples_split= 2, 
                               min_samples_leaf= 1, max_features= 'sqrt', bootstrap= True, oob_score= False,
                                n_jobs= -1, random_state= 22, verbose= 0)


hyp_grid=    'n_estimators': range(10,200, 10),
            'criterion': ['gini', 'entropy'],
            'max_depth': range(4,10), 
            'min_samples_split': range(8, 15), 
            'min_samples_leaf': range(4, 10)}


gscv_rf_model =GridSearchCV(rf__clf, hyp_grid= hyp_grid, n_jobs= -1, cv= 5)
gscv_rf_model.fit(x_train__, y_train__)

4

In [ ]:
rf__clf= gscv_rf_model.best_estimator_
rf__clf.fit(x_train__, y_train__)

In [ ]:
# This is training evaluations

y_pred_train=rf__clf.predict(x_train__)
cnf_matrix=confusion_matrix(y_train__, y_pred_train)
print("confusion Matrix of Training Evaluations:\n", cnf_matrix)
print("*"* 40)
accuracy=accuracy_score(y_train__, y_pred_train)
print("Accuracy:", accuracy )
print("*"*20)
clf_report=classification_report(y_train__, y_pred_train)
print("Classificaiton report:\n", clf_report)

# This is testing evaluations

y_pred_test=rf__clf.predict(x_test__)
cnf_matrix=confusion_matrix(y_test__, y_pred_test)
print('Confusion Matrix of Testing Evaltuations: \n', cnf_matrix)

print("*"*40)
accuracy=accuracy_score(y_test__, y_pred_test)
print("Accuracy:", accuracy)
print("*"*40)

clf_report=classification_report(y_test__, y_pred_test)
print("classification report:\n", clf_report)

In [ ]:
rf__clf.predict_proba[x_test__][:,1]

## Final Accuracy of Random Forest Model is :   

#### Lasso 

In [ ]:
lasso_clf_model =Lasso(alpha= 0.1)
lasso_clf_model.fit(x_train__, y_train__)
print(lasso_clf_model.score(x_train__, y_train__))
print(lasso_clf_model_coef_)
print('*'*20)
s1=pd.Series(lasso_clf_model_coef_, index=  x_loaded.columns)
s1.plot(kind= 'barh')

In [ ]:
lasso_clf_model =Lasso(alpha= 1.0)
lasso_clf_model.fit(x_train__, y_train__)     #Check on alpha various parameters
print(lasso_clf_model.score(x_train__, y_train__))
print(lasso_clf_model_coef_)
print('*'*20)
s1=pd.Series(lasso_clf_model_coef_, index=  x_loaded.columns)
s1.plot(kind= 'barh')

In [ ]:
lasso_clf_model =Lasso(alpha= 2.0)
lasso_clf_model.fit(x_train__, y_train__)
print(lasso_clf_model.score(x_train__, y_train__))
print(lasso_clf_model_coef_)
print('*'*20)
s1=pd.Series(lasso_clf_model_coef_, index=  x_loaded.columns)
s1.plot(kind= 'barh')

In [ ]:
lasso_clf_model =Lasso(alpha= 2.4)
lasso_clf_model.fit(x_train__, y_train__)
print(lasso_clf_model.score(x_train__, y_train__))
print(lasso_clf_model_coef_)
print('*'*20)
s1=pd.Series(lasso_clf_model_coef_, index=  x_loaded.columns)
s1.plot(kind= 'barh')  

#But wanted prior accuracy alpha want to chek for ridge and lasso

### Ridge

In [ ]:
ridge_clf_model =Ridge(alpha= 2.4)
ridge_clf_model.fit(x_train__, y_train__)
print(ridge_clf_model.score(x_train__, y_train__))
print(ridge_clf_model_coef_)
print('*'*20)
s1=pd.Series(ridge_clf_model_coef_, index=  x_loaded.columns)
s1.plot(kind= 'barh')  
 #Butwanted prior accuracy alpha want to chek for ridge and lasso

In [ ]:
training_data_r2_values=[]
testing_data_r2_values=[]

for i in np.arange(0,1,5,1.0):
    ridge_clf_model= Ridge(alpha= i)
    ridge_clf_model.fit(x_train__, y_train__)
    train_r2=ridge_clf_model.score(x_train__, y_train__)
    test_r2=ridge_clf_model.score(x_test__, y_test__)
    training_data_r2_values.append(train_r2)
    testing_data_r2_values.append(test_r2)
    

In [ ]:
alpha_range=np.arange(0.1, 5, 1.0)
plt.plot(alpha_range, training_data_r2_values)
plt.plot(alpha_range, testing_data_r2_values)

### Feature Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector


forward_sfs= SequentialFeatureSelector(estimator= rf__clf, n_features_to_select= 6, direction= 'forward', 
                         cv= 5, n_jobs= -1)

forward_sfs.fit(x_loaded,y_loaded)

In [ ]:
forward_sfs.get_feature_names_out()

In [ ]:
from mlxtend.feature_selction import SequentialFeatureSelector
forward_sfs= SequentialFeatureSelector(estimator= rf__clf, n_features_to_select= 6, direction= 'Forward',
                                      cv= 5, scoring= None,  n_jobs= -1)
forward_sfs.fit(x_train__,y_train__)
pd.DataFrame(forward_sfs.subsets_).T

In [ ]:
from mlxtend.feature_selction import SequentialFeatureSelector


forward_sfs= SequentialFeatureSelector(estimator = rf__clf, k_features= 6, direction= 'forward',scoring= True, 
                                              cv= 5, n_jobs= -1)
forward_sfs.fit(x_train__,y_train__)
pd.DataFrame(forward_sfs.subsets_).T

In [ ]:
backward_sfs.get_support()

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector

backward_sfs= SequentialFeatureSelector(estimator = rf__clf, k_features= 6, direction= 'backward',scoring= True, 
                                              cv= 5, n_jobs= -1)
backward_sfs.fit(x_train__,y_train__)
backward.sfs.get_feature_names_out()


In [ ]:
pd.DataFrame(forward_sfs.subsets_).T

In [ ]:
from mlxted.feature_selction import ExhaustiveFeatureSelector

efs= ExhaustiveFeatureSelector(estimator = rf__clf, min_features= 5, socring= 'accuracy', max_features=7, print_progress= True, 
                                              cv= 5, n_jobs= -1)
efs.fit(x_train__,y_train__)


In [ ]:
efs.best_feature_names_

In [ ]:
efs.best_score_

In [ ]:
pd.DataFrame(efs.subset_)T.

In [ ]:
from sklean.feature_selection import RFE


rfs= RFE(estimator = rf__clf,n_features_to_select =5, verbose = 1)
rfs.fit(x_train__,y_train__)

rfs.get_feature_names_out()

## Ensemble Techniques

1. Bagging-- i.e RF  #Done
2. Boostring -- Adaptive Boosting Chosen here   #Doing

## Adaptive Boosting (AdaBoost )

In [4]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:

x_train, x_test, y_train, y_test= train_test_split(x_loaded, y_loaded, test_size= 0.2, random_state=22, stratify= y_loaded)
print(y_train.value_counts())
print('*'*20)





adb_clf= AdaBoostClassifier(estimator= None, n_estimators= 50, learning_rate= 1.0, random_state=22)

adb_clf.fit(x_train, y_train)

In [ ]:
abd_clf.feature_importances_

In [ ]:
s1= pd.Series(abd_clf.feature_importances_, index= x_loaded.columns)
s1.plot(kind= 'barh')

In [ ]:
# This is training evaluations without Scalling

y_pred_train=abd_clf.predict(x_train)
cnf_matrix=confusion_matrix(y_train, y_pred_train)
print("confusion Matrix of Training Evaluations:\n", cnf_matrix)
print("*"* 40)
accuracy=accuracy_score(y_train, y_pred_train)
print("Accuracy:", accuracy )
print("*"*20)
clf_report=classification_report(y_train, y_pred_train)
print("Classificaiton report:\n", clf_report)

# This is testing evaluations witout scalling

y_pred_test=abd_clf.predict(x_test)
cnf_matrix=confusion_matrix(y_test, y_pred_test)
print('Confusion Matrix of Testing Evaltuations: \n', cnf_matrix)

print("*"*40)
accuracy=accuracy_score(y_test, y_pred_test)
print("Accuracy:", accuracy)
print("*"*40)

clf_report=classification_report(y_test, y_pred_test)
print("classification report:\n", clf_report)

### Hyper-Parameter Tunning   [without Scalling]

In [ ]:
abd_clf_ =AdaBoostClassifier(n_estimator= 50, learning_rate= 1.0, random_state=22)
hyp_grid= {
    'n_estimators': range(10,70,5)
    'learning_rate': np.arange(0.01, 1, 0.01)
}


gscv_abd_clf_ =GridSearchCV(abd_clf_, hyp_grid, cv= 5, n_jobs= -1, verbose= 1)
gscv_abd_clf_.fit(x_train, y_train)

In [ ]:
adb_clf_= gscv_abd_clf.best_estimator_
abd_clf

In [ ]:
adb_clf_= AdaBoostClassifier(n_estimator= 50, learning_rate= 1.0, random_state=22)
adb_clf_= gscv_abd_clf.best_estimator_
adb_clf_.fit(x_train, y_train)

In [ ]:
 # Training data evaluation without scalling 
 y_pred_train = adb_clf_.predict(x_train)
 cnf_matrix = confusion_matrix(y_train, y_pred_train)
 print("Confusion Matrix :\n", cnf_matrix)
 print('*'*40)
 accuracy = accuracy_score(y_train, y_pred_train)
 print("Accuracy :",accuracy)
 print('*'*40)
 clf_report = classification_report(y_train, y_pred_train)
 print("Classification Report :\n", clf_report

print('*'*50)

# Testing data evaluation
 y_pred_test = adb_clf_.predict(x_test)
 cnf_matrix = confusion_matrix(y_test, y_pred_test)
 print("Confusion Matrix :\n", cnf_matrix)
 print('*'*40)
 accuracy = accuracy_score(y_test, y_pred_test)
 print("Accuracy :",accuracy)
 print('*'*40)
 clf_report = classification_report(y_test, y_pred_test)
 print("Classification Report :\n", clf_report